# IMPORTAR LIBRERIAS

In [1]:
import requests
import pandas as pd
import json

C:\Users\OT\AppData\Local\Temp\ipykernel_10648\3960648340.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# PRUEBAS API

Para poder lanzar la API en local realizamos lo siguiente:
- Creamos nuestro archivo de la API .py, en nuestro caso: app.py.
- Navegar al directorio donde se encuentra app.py y ejecutar el archivo con python.
- Una vez que la API esté corriendo, enviar datos a ella con el siguiente bloque para probar su funcionamiento.

Como evalua la API con un ejemplo: 

Actualmente tenemos cada serie de alimento de 1999 hasta 2023, y los modelos entrados hasta todo el conjunto de datos y con el pronóstico de 2024. Cuando finalizemos el año 2024, meteremos los datos de 2024, y evaluaremos la métrica RMSE con este 2024 como test para ver si mantiene el rango RMSE del pronóstico del año 2023, con una diferencia menor al 5%, si no lanzará un mensaje para revaluar el modelo. 
Si esta por debajo del 5%, reentrena el modelo con todo el conjunto y lanza el pronóstico de 2025. 

## OBTENER FECHAS:

Obtener fechas traspasando el alimento de la serie que queremos introducir y pronosticar. 

In [2]:
# URL del endpoint de tu API
url = 'http://127.0.0.1:5000/obtener_fechas'

# Parámetro de alimento que quieres probar
alimento = 'T.FRUTAS FRESCAS'

# Hacer la solicitud GET a la API
response = requests.get(url, params={'alimento': alimento})

# Verificar el estado de la respuesta
if response.status_code == 200:
    # Si la respuesta es exitosa (código 200), imprimir el contenido
    data = response.json()
    print("Respuesta de la API:", data)
else:
    # Si hay un error, imprimir el código de estado y el mensaje de error
    print("Error:", response.status_code)
    print("Mensaje de error:", response.text)

Respuesta de la API: {'fechas': ['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01', '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01']}


## PRONOSTICAR CONSUMO PER CÁPITA NUEVO AÑO

Aquí debemos enviarle  el alimento seleccionado de la serie a predecir con las fechas obtenidas anteriormente. Además enviar los nuevos datos del año finalizado: consumo, gasto y precio medio, y las variables exógenas del año nuevo a pronosticar.

In [3]:
# Crear datos de prueba
datos = {
    'fechas':['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
               '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01'],
    'alimento': 'T.FRUTAS FRESCAS',
    'consumo': [
        1.933141, 2.042574, 2.073673, 2.118096,
        2.158674, 2.053273, 2.122503, 1.831925,
        1.884250, 2.144403, 2.134640, 2.108376
    ],
    'gasto':  [
        11.944193, 11.459124, 13.434974, 12.603300,
        14.900921, 15.251681, 15.097949, 13.834472,
        14.234611, 13.046981, 12.188683, 11.635794
    ],
    'precio': [
        1.933141, 2.042574, 2.073673, 2.118096,
        2.158674, 2.053273, 2.122503, 1.831925,
        1.884250, 2.144403, 2.134640, 2.108376
    ],
    'gasto_exogenas': [
        11.362245, 11.265531, 12.669824, 11.973228,
        13.676440, 15.541552, 14.998620, 13.420511,
        14.188105, 13.734665, 11.960396, 11.252989
    ],
    'precio_exogenas': [
        1.779500, 1.857111, 1.918180, 1.914666,
        2.016780, 2.096312, 1.814827, 1.799019,
        2.000987, 2.119147, 2.021288, 1.986037
    ]
}

# URL de la API Flask
url = 'http://127.0.0.1:5000/enviar'

# Enviar la solicitud POST
response = requests.post(url, json=datos)

# Imprimir la respuesta
print(response.json())

{'alimento': 'T.FRUTAS FRESCAS', 'fecha': ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01', '2025-06-01', '2025-07-01', '2025-08-01', '2025-09-01', '2025-10-01', '2025-11-01', '2025-12-01'], 'message': 'Datos recibidos, serie actualizada, y pronóstico calculado correctamente', 'prediccion': [11.362244999999993, 11.265530999999994, 12.669823999999995, 11.973227999999994, 13.676439999999992, 15.54155199999999, 14.998619999999994, 13.420510999999992, 14.188104999999995, 13.734664999999993, 11.960395999999992, 11.252988999999994], 'rmse_new': 1e-08, 'rmse_old': 2.5096768003166226e-06, 'status': 'success'}
